宮沢賢治の小説を学習データに使い、賢治風の返事ができるようにモデルを訓練。モデル同士の対話をさせる。動作テスト。

In [1]:
import pickle

with open('kana_chars.pickle', mode='rb') as f:
    chars_list = pickle.load(f)
print(chars_list)

['\t', '\n', '、', '。', '々', 'ぁ', 'あ', 'ぃ', 'い', 'ぅ', 'う', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'ゎ', 'わ', 'ゐ', 'ゑ', 'を', 'ん', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'エ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'テ', 'デ', 'ト', 'ド', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ャ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ヮ', 'ワ', 'ヰ', 'ヱ', 'ヲ', 'ン', 'ヴ', '・', 'ー', '？']


In [2]:
#文章をone-hot表現に変換する関数を設定

import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
n_char = len(chars_list)
max_length_x = 128

# 文章をone-hot表現に変換する関数
def sentence_to_vector(sentence):
    vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)
    for j, char in enumerate(sentence):
        vector[0][j][char_indices[char]] = 1
    return vector

In [4]:
#encoderのモデル、およびdecoderのモデルを読み込み、返答を作成するための関数を設定

from keras.models import load_model

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

def respond(input_data, beta=5):
    state_value = encoder_model.predict(input_data)
    y_decoder = np.zeros((1, 1, n_char))  # decoderの出力を格納する配列
    y_decoder[0][0][char_indices['\t']] = 1  # decoderの最初の入力はタブ。one-hot表現にする。

    respond_sentence = ""  # 返答の文字列
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power)) 
        next_char = indices_char[next_index]  # 次の文字
        
        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break  # 次の文字が改行のとき、もしくは最大文字数を超えたときは終了
            
        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))  # 次の時刻の入力
        y_decoder[0][0][next_index] = 1

        state_value = h  # 次の時刻の状態

    return respond_sentence

In [5]:
#コーパスに無い文章を使って、対話生成の検証

sentences = ["けんじさん、こんにちは。",
             "カムパネラってしってますか？",
             "ああ、おなかすいた。",
             "きょうはいいてんきですね。",
             "すきなたべものは、なんですか。",
             "きょうは、さんぽにいきました。",
             "パソコンかスマホは、つかったことありますか？"]

for sentence in sentences:
    vector = sentence_to_vector(sentence)
    print("Input:", sentence)
    print("Response", respond(vector))
    print()

Input: けんじさん、こんにちは。
Response せいねんはいいました。

Input: カムパネラってしってますか？
Response ジョバンニはまるであかいひかりをみていましたが、そのとき、あかいはかせは、そのとき、それからはしっていきました。

Input: ああ、おなかすいた。
Response ああ、こんどはまたさぶろうはすこしきょうしつのなかをみていましたが、そのときは、あかいはかせは、そのままそのなかにはいってきて、いちめんにおおきなおおきなきがして、それから、そのとき、それから、おおきなきがして、そのとき、ちいさなこどものじょうに、そのとき

Input: きょうはいいてんきですね。
Response ぼくはこんどはまたさぶろうはすこしかおをあかくしていいました。

Input: すきなたべものは、なんですか。
Response といいました。

Input: きょうは、さんぽにいきました。
Response ホモイはおもわずわらいました。

Input: パソコンかスマホは、つかったことありますか？
Response こういちはまだあかるくなって、まるでいっぽんのきをいいました。



In [6]:
#モデル同士の会話

response_a = "こんにちは。"
response_b = ""
for i in range(100):
    print("賢治A:", response_a)
    print()  
    vector_a = sentence_to_vector(response_a)
    
    response_b = respond(vector_a)
    print("賢治B:", response_b)
    print()
    vector_b = sentence_to_vector(response_b)
    
    response_a = respond(vector_b)

賢治A: こんにちは。

賢治B: ブドリはしばらくあおくなっていましたが、そのときは、あかいはかせは、あかいはかせは、そのときあしをみました。

賢治A: そしてそのときは、あかいはかせは、そのときあるいていきました。

賢治B: そしてそのときは、あかいはかせは、いちめんにおおきなかんがえていました。

賢治A: そのとき、あかいはかせは、おおきなこえでした。

賢治B: ホモイはそれをみました。

賢治A: うまいぞ。

賢治B: もうここはいちぺんにいってるんだ。

賢治A: それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それか

賢治B: それから、そこではいっているのです。

賢治A: それから、そのときは、ああ、ぼくはもうみんなはないんだ。

賢治B: おれのところへいってるんだ。

賢治A: よろしい。

賢治B: はじめ。

賢治A: かしわのきは、なにかしかたないというようにおもいました。

賢治B: どうもありがとう。

賢治A: どうもあんまりおまえたちはいったんだ。

賢治B: どうだ。

賢治A: ここはいいました。

賢治B: それから、そのとき、おまえたちはあるいているんだ。

賢治A: ぼくは、いっしょうけんいのちになって、それからあのきりのようなかおをして、そのときぼくはもうしろいことをかっているんだ。

賢治B: ところがそのこういちはきょうしつのなかをとりました。

賢治A: そしてそのときは、あのかわのみずのなかには、いっぽんのおとうさんが、まっくろなのはらに、あかいのはらのおとこが、いっぽんのおとうさんがききました。

賢治B: そしてそのときは、あかいはこんなにおとこのこがでていってしまいました。

賢治A: そしてそのとき、そのかわのみずのなかには、あかいはたっていました。

賢治B: そしてそのときは、あかいはかせが、そのとき、きつねのうえを、あかいはあのあかりを、あかいのでした。

賢治A: そしてそのおとこは、そのとき、それから、そのとき、それから、こういちはきょうもありませんでした。

賢治B: そのとき、あか

賢治A: そしてそのままそのにんのおとうさんがききました。

賢治B: そしてそのときは、そのにんのたいていきました。

賢治A: そしてそのときは、あのあかりを、あかいのでした。

賢治B: そのとき、ちょうどそのとき、おかあさんが、いっしょについていきました。

賢治A: そしてそのこはまるでかおをあかくしながら、そのときあしをかけて、そのときあたまをあけて、それから、そのときは、あかいはしらないように、あかいしゃくにさわりました。

賢治B: そしてそのにんは、ちょうどおまえたちのほうへはいっていました。

賢治A: そしてそのときは、あかいはかせは、あかいのはらをあけて、そのときあたまをかけて、そのときあしをたてて、そのときあたまをかけて、そのかわのみずのなかに、おおきなこえでした。

賢治B: そしてそのときは、あのかわりのはらのはなしのようなきがして、そのかわのみずのなかにはいってきて、いちめんにいいました。

賢治A: そのとき、きれいなかたちになって、そのかわのみずのなかで、いっしょうけんいのちになって、いままでのぼっていきました。

賢治B: そしてそのときは、あかいはあのあかりを、かおをあけて、そのときあたまをかけて、そこでしたのです。

賢治A: そしてそのときは、どうもありがとうございます。

賢治B: かぜのまたさぶろうはすこしおおきなかおをみていました。

賢治A: そして、そのときは、そのときは、そのとき、きつねが、そのとき、きれいになって、いきなり、そのときは、あのあとのかおをみているのでした。

賢治B: そしてそのときは、あかいはかせが、そのときあたまをくわえて、それから、そのとき、そのとききしゃのおとうさんが、いっぽんのかおをみていました。

賢治A: そしてそのときは、あかいはいってきました。

賢治B: そのとき、あかいはかせは、あのかわのみずのなかにはいってきたのです。

賢治A: そしてそのときは、あのまえにあるいていました。

賢治B: そしてそのときは、あかいはかせは、そのとき、きっとみえました。

賢治A: そしてそのおとこは、なにかのかんがえていました。

賢治B: そのとき、そのとき、きつねが、そのかわりになって、そのかわのみずのなかにはしっているのでした。

賢治A: そしてそのときは、ああ、ぼくはこんどはま